## Data Preparation

Download files from https://sites.betfair.com.au/dap/ > Greyhounds > Month by Month Pro

Decompress under `flumine-example/data/`
eg: flumine-example/data/PRO/2023/Jan/...

In [1]:
import logging

from flumine import FlumineSimulation, clients
from betfairlightweight.filters import streaming_market_filter, streaming_market_data_filter

import os
import sys
module_path = os.path.abspath(os.path.join('..', 'src'))
if module_path not in sys.path:
    sys.path.append(module_path)
from historical_files_parser import parse_market_files
from example_strategy import ExampleStrategy

logger = logging.getLogger()
logger.setLevel('INFO')

# Silence flumine
flumine_logger = logging.getLogger('flumine')
flumine_logger.setLevel('WARNING')
base_flumine_logger = logging.getLogger('flumine.baseflumine')
base_flumine_logger.setLevel('WARNING')
betfairlightweight_logger = logging.getLogger('betfairlightweight')
betfairlightweight_logger.setLevel('WARNING')
betfairlightweight_logger = logging.getLogger('betfairlightweight.streaming.listener')
betfairlightweight_logger.setLevel('ERROR')
betfairlightweight_logger = logging.getLogger('betfairlightweight.streaming.stream')
betfairlightweight_logger.setLevel('ERROR')

In [2]:
if __name__ == "__main__":
    logging.info('Starting Betfair Trading Bot in backtesting mode')

    client = clients.SimulatedClient(username="backtest", min_bet_validation=False, transaction_limit=5000)
    framework = FlumineSimulation(client=client)

    # Load strategy
    markets = parse_market_files(
        data_path='../data/PRO',
        market_definition_filters={'bettingType': 'ODDS', 'marketType': 'WIN', 'countryCode': 'AU'},
        include_years=["2023"],
        include_months=["Jan"],
        include_days=["1"] # To test a specific day
    )

    logging.info(f"### Backtesting {len(markets)} market")
    
    example_strategy = ExampleStrategy(
        market_filter={
            "markets": markets,
            "listener_kwargs": {"inplay": False, "seconds_to_start": 60}
        },
        market_data_filter=streaming_market_data_filter(fields=["EX_ALL_OFFERS", "EX_TRADED", "EX_MARKET_DEF"]),
        max_order_exposure=10000,
        max_selection_exposure=5000,
        max_trade_count=2,
        max_live_trade_count=2,
        conflate_ms=1000,
        context={
            'staking_strategy': 'offer',
            'stake': 5,
            'margin': 0.1,
            'min_back_price': 1,
            'max_back_price': 100,
            'min_lay_price': 1,
            'max_lay_price': 100,
            'seconds_to_start': 60
        }
    )
    framework.add_strategy(example_strategy)

    framework.run()

INFO:root:Starting Betfair Trading Bot in backtesting mode
INFO:root:### Backtesting 96 market
INFO:root:Added strategy 'ExampleStrategy' to framework
INFO:root:Starting strategy 'ExampleStrategy'
INFO:root:Generated random probabilities for maket=1.208138899, probabilities=[0.24637834 0.01295797 0.07460724 0.21848847 0.14747029 0.30009769]
INFO:root:TOTAL market_id=1.208138899	back=6.25	lay=15.0	market_pnl=19.76,	market_commission=1.49
INFO:root:Generated random probabilities for maket=1.208138901, probabilities=[0.23413047 0.12012892 0.05523587 0.39341189 0.19709285]
INFO:root:TOTAL market_id=1.208138901	back=13.5	lay=15.0	market_pnl=26.5,	market_commission=2.0
INFO:root:Generated random probabilities for maket=1.208138903, probabilities=[0.1237373  0.2692641  0.01908869 0.33520374 0.17464119 0.07806498]
INFO:root:TOTAL market_id=1.208138903	back=-10.0	lay=-20.12	market_pnl=-30.12,	market_commission=0
INFO:root:Generated random probabilities for maket=1.208138905, probabilities=[0.12

KeyboardInterrupt: 